# Lecture 01

#### All the imports here

In [9]:
import numpy as np
import scipy.misc as sc
import matplotlib.pyplot as plt

### Task 2.1
##### 1. upper bound for the number of nodes in the complete tic tact toe game tree (starting with an empty board and player X to move)

$$ supper \, upper \, bound = \sum_{i=1}^{9} \prod_{j=i}^{9} {j \choose 1} $$



In [10]:
sum = 0
for i in range(0, 10):
    product = 1
    for j in range(i, 9):
        product *= sc.comb(j, 1, True)
    sum += product
print 'Supper upper bound is: ' + str(sum)

Supper upper bound is: 109601


In [16]:
class TicTacToe:
    """ tic tac toe implementation """
    def __init__(self):
        # relate numbers (1, -1, 0) to symbols ('x', 'o', ' ')
        self.symbols = {1: 'x', -1: 'o', 0: ' '}
        self.result = []
        # x_board and o_board hold the win information after 1000 runs
        self.x_board, self.o_board = np.zeros((3, 3), dtype=float), np.zeros((3, 3), dtype=float)
        # hold all the three in a line information
        self.Three_in_a_Row = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8],
                                        [0, 3, 6], [1, 4, 7], [2, 5, 8],
                                        [0, 4, 8], [2, 4, 6]])
        # heuristic array
        self.Heuristic_Array = [[0, -10, -100, -1000],
                                [10, 0, 0, 0],
                                [100, 0, 0, 0],
                                [1000, 0, 0, 0]]
        self.nodeDict, self.succDict = {}, {}
        self.strategy = ''

    def move_still_possible(self, S):
        return not (S[S == 0].size == 0)

    def move_was_winning_move(self, S, p):
        if np.max((np.sum(S, axis=0)) * p) == 3:
            return True

        if np.max((np.sum(S, axis=1)) * p) == 3:
            return True

        if (np.sum(np.diag(S)) * p) == 3:
            return True

        if (np.sum(np.diag(np.rot90(S))) * p) == 3:
            return True

        return False

    # evaluation function
    def evaluation(self, S, p):
        opponent = p * -1
        t = 0
        for i, items in enumerate(self.Three_in_a_Row):
            players, others = 0, 0
            for j, _ in enumerate(items):
                piece = S[self.Three_in_a_Row[i][j] / 3][self.Three_in_a_Row[i][j] % 3]
                if piece == p:
                    players += 1
                elif piece == opponent:
                    others += 1
            t += self.Heuristic_Array[players][others]
        return t


class MinMax:

    def __init__(self, game='', game_name='', player='', level='', state=''):
        self.game = game
        self.game_name = game_name
        self.player = player
        self.level = level
        self.state = state
        self.players = []
        self.nodeDict, self.nodeSuccDict, self.nodeUtilDict, self.nodeMinMaxDict = {}, {}, {}, {}

    def min_node_util(self, node):
        if node in self.nodeUtilDict:
            self.nodeMinMaxDict[node] = self.nodeUtilDict[node]
            return self.nodeMinMaxDict[node]
        mmv = np.inf
        for s in self.nodeSuccDict[node]:
            mmv = min(mmv, self.max_node_util(s))
        self.nodeMinMaxDict[node] = mmv
        return mmv

    def max_node_util(self, node):
        if node in self.nodeUtilDict:
            self.nodeMinMaxDict[node] = self.nodeUtilDict[node]
            return self.nodeMinMaxDict[node]
        mmv = -np.inf
        for s in self.nodeSuccDict[node]:
            mmv = max(mmv, self.min_node_util(s))
        self.nodeMinMaxDict[node] = mmv
        return mmv

    def next_step_ttt(self, player, state, succ):
        rs, cs = np.where(state == 0)
        for j in range(rs.size):
            ss_copy = np.copy(state)
            ss_copy[rs[j], cs[j]] = player

            newnode = max(self.nodeDict.keys()) + 1
            self.nodeDict[newnode] = ss_copy

            succ.append(newnode)

    def next_step_connect4(self, player, state, succ):
        for i in range(7):  # enumerate all legal moves from this state
            if self.game.is_valid_move(state, i):  # if column is a legal move
                ss_copy = np.copy(state)
                self.game.make_move(ss_copy, player, i)

                newnode = max(self.nodeDict.keys()) + 1
                self.nodeDict[newnode] = ss_copy

                succ.append(newnode)

    def build_tree(self, state, player, node, level):
        succ = []

        # if state is not terminal: switch player & compute successors
        if level == 0 or self.game.move_was_winning_move(state, player):
            self.players.append(player)
            self.nodeUtilDict[node] = self.game.evaluation(state, player)
        
        elif not self.game.move_still_possible(state):
            self.players.append(0)  # no one win, game ends in a draw

        else:
            player *= -1
            if self.game_name is 'ttt':
                self.next_step_ttt(player, state, succ)
            elif self.game_name is 'connect_four':
                self.next_step_connect4(player, state, succ)
            else:
                raise ValueError("Game name is not as expected!")

        self.nodeSuccDict[node] = succ

        for s in succ:
            self.build_tree(self.nodeDict[s], player, s, level - 1)

    def run_min_max(self):

        # build tree
        node = 0
        self.nodeDict[node] = self.state
        state, player, level = self.state, self.player, self.level
        self.build_tree(state, player, node, level)
        f = open('output.txt', 'w')
        f.write('Number of nodes: ' + str(len(self.nodeDict)) + '\n') 
        f.write('Number of draw: ' + str(self.players.count(0)) + '\n')
        f.write('Number of x: ' + str(self.players.count(1)) + '\n')
        f.write('Number of o: ' + str(self.players.count(-1)) + '\n')
        mmv = self.max_node_util(node)
        f.write(str(self.nodeSuccDict))
        return next(self.nodeDict[succ] for succ in self.nodeSuccDict[node] if self.nodeMinMaxDict[succ] is mmv)  

In [17]:
'''
ttt = TicTacToe()
state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])
player = -1
level = np.inf
minmax = MinMax(ttt, 'ttt', player, level, state)
minmax.run_min_max()
'''

"\nttt = TicTacToe()\nstate = np.array([\n    [0, 0, 0],\n    [0, 0, 0],\n    [0, 0, 0]\n])\nplayer = -1\nlevel = np.inf\nminmax = MinMax(ttt, 'ttt', player, level, state)\nminmax.run_min_max()\n"

### Task 2.2

In [20]:
nodeDict = {0, 1 ,2 ,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
nodeSuccDict = {0: [1, 2, 3, 4, 5], 1: [6, 7, 8, 9], 2: [10, 11], 3: [12, 13], 4: [14, 15, 16], 5: [17, 18, 19]}
nodeUtilDict = {6: 15, 7: 20, 8: 1, 9: 3, 10: 3, 11: 4, 12: 15, 13: 10, 14: 16, 15: 4, 16: 12, 17: 15, 18: 12, 19: 8}
nodeMinMaxDict = {}

def maxNodeUtil(node):
    if node in nodeUtilDict:
            nodeMinMaxDict[node] = nodeUtilDict[node]
            return nodeMinMaxDict[node]
    mmv = -np.inf
    for s in nodeSuccDict[node]:
            mmv = max(mmv, minNodeUtil(s))
    nodeMinMaxDict[node] = mmv
    return mmv

def minNodeUtil(node):
    if node in nodeUtilDict:
            nodeMinMaxDict[node] = nodeUtilDict[node]
            return nodeMinMaxDict[node]
    mmv = np.inf
    for s in nodeSuccDict[node]:
            mmv = min(mmv, maxNodeUtil(s))
    nodeMinMaxDict[node] = mmv
    return mmv

if __name__ == '__main__':
    print "----"
    print maxNodeUtil(0)
    print "----"

----
10
----
